# Parameterize Pulse Sequences in Bloqade

In the introduction we discussed how to build bloqade programs but this only scratches the surface of what you can do.
In Bloqade we allow you to specify the value of your program as a parameter that is unknown at the time of building the
program. This is useful for performing parameter scans or for using hybrid quantum-classical algorithms. In this
tutorial we will use the same example as in the previous tutorial, but we will perform an optimization algorithm to
find the optimal final detuning value for the adiabatic ramp. Let is first import the necessary modules and define the
geometry and pulse sequence.

In [3]:
from bloqade import piecewise_linear, rydberg_h, var
from bloqade.atom_arrangement import Square
import numpy as np

from bokeh.io import output_notebook 

output_notebook()

Loading BokehJS ...

Note that we now also import `var` from bloqade which is a function that allows us to define a variable that can be 
specified when running the program. to create a variable simply call `var` with a name specified as a string. Any reference 
to that variable name within the program will be replaced with the value specified at run time. There are three ways 
to specify the value of a variable but for this example we will focus on `args` which allows you to pass the value as 
an argument to the `run` method.

In [4]:

# geometry
rng = np.random.default_rng(1234)
atom_pos = (
    Square(4, lattice_spacing=4.5)
    .apply_defect_density(0.3, rng=rng)
    .remove_vacant_sites()
)

# variables
final = var("final")

# dynamics
durations = [0.15, 3.7, 0.15]
delta_MHz = [-13.0, -13.0, final, final]
omega_MHz = [0.0, 2.5, 2.5, 0.0]

Delta = piecewise_linear(durations, [x * 2 * np.pi for x in delta_MHz])
Omega = piecewise_linear(durations, [x * 2 * np.pi for x in omega_MHz])

# create Hamiltonian
program = rydberg_h(
    atom_pos, detuning=Delta, amplitude=Omega, phase=None, args=["final"]
)

The only changes we made to the original program are:
1. We use the `var` function to define the final detuning value as a variable. 
2. We pass the variables string as an argument to the `rydberg_h` function using the `args` keyword argument.
   This tells bloqade that the variable will be specified at run time.
Note that variables have basic support for basic arithmetic operations, `+`, `-`, `*`, `/`, `-`. The resuting 
expresssions weill be evaluated when specifying the value of the variable. For example:
```python
final = var("final")
final_2 = final + 1
final_3 = final_2 * 2
final_4 = final_3 / 2
final_5 = final_4 - 1
```
All these expressions can be used in  your program and will be evaluated when you specify the value of `final`.

## Running the job
Now that we have a program with a parameter we can run the job. Its similar to running the jobs in the previous
tutorial but we need to pass the value of the parameter as an argument to the `run` method. The order of the arguments
passed to the `run` method should match the order of the strings passed to the `args` keyword argument of the `rydberg_h`

In [5]:

solver_options = dict(
    atol=1e-7, rtol=1e-4, interaction_picture=True, solver_name="dop853"
)
bloqade_program = program.bloqade.python()
result = bloqade_program.run(100, args=(3.0,), **solver_options)
result.report().rydberg_densities().to_numpy().sum()

3.2800000000000002

## Emulating a hybrid quantum-classical algorithm
Now that we have a way to run our parameterized pulse sequence with arguments we can use it to emulate a hybrid
quantum-classical algorithm. In this example we will use the Nelder-Mead algorithm to find the optimal final
detuning value for the adiabatic ramp. We will use the `scipy.optimize.minimize` function to perform the
optimization. This function takes a function to minimize and an initial guess for the parameters.

the cost function is given by:

In [10]:
from bloqade.emulate.ir.state_vector import StateVector, RydbergHamiltonian
from bloqade import get_capabilities

def cost(params, bloqade_program):
    result = bloqade_program.run(100, args=(params[0],), **solver_options)
    rydberg = result.report().rydberg_densities().to_numpy().sum()
    print(f"final detuning: {params[0]}, rydberg: {rydberg}")
    return -rydberg



c = get_capabilities()
delta_max = float(c.capabilities.rydberg.global_.detuning_max) / (2 * np.pi)


In [8]:
from scipy.optimize import minimize

# we need to pass the program as a keyword argument
# because the cost function only takes one argument
# but the minimize function passes two arguments
# to the cost function


result = minimize(
    cost, 4, args=(bloqade_program,), method="Nelder-Mead", bounds=[(0, delta_max)]
)

NameError: name 'cost' is not defined

# Advanced useage: run_callback

When doing the simulation we do not have to use a shot based average to calculate the cost function. We can use the `run_callback` method to run the program. The callback function is given access to the full quantum state so we can calculate the expectation value of the number of Rydberg excitations at the end of the pulse sequence numerically exactly. An advantage to doing this over a shots based average is that the cost function is actually smooth and we can use gradient based optimization algorithms. This is not possible with the shots based average because the cost function is not smooth and the gradient is not well defined.


In [11]:
def cost_advanced(params, bloqade_program):
    def measure_density(state: StateVector, metadata, ham: RydbergHamiltonian):
        # 0: ground state, 1: Rydberg state
        n = np.array([[0, 0], [0, 1]])
        n_atoms = state.space.n_atoms

        # calculate average and variace of the energy
        # at the end of the simulation. Minimize the average energy
        # will also obtrain the MIS
        # E, dE = ham.average_and_variance(state)

        return sum(state.local_trace(n, i).real for i in range(n_atoms))

    (rydberg,) = bloqade_program.run_callback(
        measure_density, program_args=tuple(params), **solver_options
    )
    print(f"final detuning: {params[0]}, rydberg: {rydberg}")
    return -rydberg

We are using the `run_callback` method to run the program and measure the average number of Rydberg excitations.
This will be better than using the shots based average because the expectation values are numerically exact. You can try
using the `run` method and the `shots` argument to see the difference in the results.

The `state` object passed into your callback function can be used to measure two body correlation functions, 
two body local observables as well:

```python
n = np.array([[0, 0], [0, 1]]) # density operator
nn = np.kron(n, n) # two body correlation operator
corr_0_4 = state.local_trace(nn, (0, 4))
```

In [12]:
c = get_capabilities()
delta_max = float(c.capabilities.rydberg.global_.detuning_max) / (2 * np.pi)

result = minimize(
    cost_advanced, 4, args=(bloqade_program,), method="Nelder-Mead", bounds=[(0, delta_max)]
)

final detuning: 4.0, rydberg: 3.6332788176871933
final detuning: 4.2, rydberg: 3.689623540261508
final detuning: 4.4, rydberg: 3.7396964002301942
final detuning: 4.600000000000001, rydberg: 3.7832930811556365
final detuning: 5.000000000000003, rydberg: 3.8519260218954887
final detuning: 5.400000000000004, rydberg: 3.9013131520076967
final detuning: 6.200000000000006, rydberg: 3.9614445967246183
final detuning: 7.000000000000007, rydberg: 3.98750987860161
final detuning: 8.60000000000001, rydberg: 3.9962777986747398
final detuning: 10.200000000000014, rydberg: 4.006802162599883
final detuning: 13.40000000000002, rydberg: 4.256388484157421
final detuning: 16.600000000000026, rydberg: 4.877537633814875
final detuning: 19.89436788648692, rydberg: 5.068443941347948
final detuning: 19.89436788648692, rydberg: 5.068443941348054
final detuning: 19.89436788648692, rydberg: 5.068443941347864
final detuning: 19.89436788648692, rydberg: 5.068443941347883
